## Coursera - Capstone Project for IBM Data Science Certificate
Segmenting and Clustering Neighborhoods in Toronto 


**Objective:** explore, segment, and cluster the neighborhoods in the city of Toronto


**Data Sources:** (Wiki page and shared csv files via Google Drive)
> https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

> http://cocl.us/Geospatial_data

> https://github.com/tookitooki12/Coursera_Capstone

**Table of contents:**

*   System & Data Setup
*   **Part 1** - Create initial table with 103 postal codes ('Postcode', 'Borough','Neighborhood')
*   **Part 2** - Concatinate table from part 1 with Geospacial Coordinates ('Latititude', 'Longitude')
*   **Part 3** - Generate maps to visual neighborhoods and how they cluster using geopy & folium

> **3a - general map of toronto by Postcode**

> **3b - review of venues in 'Studio District'**

> **3c - cluster analysis of district by venue**  



## System & Data Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

create pydrive for uploading of csv file 'Geospatial_Coordinates'

In [2]:
# read csv file
Geospacial_Coordinates = pd.read_csv('http://cocl.us/Geospatial_data', sep = ',') 
# examine the shape of original input data
print(Geospacial_Coordinates.shape)

(103, 3)


**Create table - step 1** use BeautifulSoup to scrape data from website:

In [3]:
#create an object with raw data from website
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [4]:
#create an object with the data from the website
soup = BeautifulSoup(website_url)

**Create table - step 2** based on the table in the website (search for table > extract to dict & create pd DataFrame)

In [5]:
#search for table
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table; #remove ';' to view output

**Create table - step 3** extract table data and create pd DataFrame

In [6]:
#extract row data to dict
row_data = []
for row in My_table.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    row_data.append(cols)

row_data; #remove ';' to view output

## Part 1 - Create initial table with 103 Postcodes ('Postcode', 'Borough', 'Neighborhood')

In [7]:
#create initial pd DataFrame
df_table = pd.DataFrame(row_data)
df_table = df_table.rename(columns={0:"Postcode",1:"Borough",2:"Neighborhood"})
df_table.head()

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [8]:
#drop the first row (index = 0), and any row where 'Bourough' = 'Not assigned'
df_table2 = df_table.copy()
df_table2 = df_table.drop([0])
df_table2 = df_table2.drop(df_table2[df_table2['Borough']=='Not assigned'].index)
df_table2 = df_table2.reset_index(drop=True)
df_table2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Create table - step 4** - data transform - if 'Neighborhood' = 'Not Assigned', then use 'Borough'

In [9]:
#check a row where 'Neighborhood' = 'Not assigned'
df_table2.loc[6]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 6, dtype: object

In [10]:
#create a new table and replace values if 'Neighborhood' = 'Not assigned' with 'Bourough'
df_table3 = df_table2.copy()

df_table3['Neighborhood'] = df_table3.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'],
    axis=1
)

#have a look at the transformed data
df_table3.loc[6]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object

**Create table - step 5** group the dataframe by Postcode & Borough and 'Join' values in 'Neighborhood'

In [11]:
df_table4 = df_table3.copy()

df_table4 = (df_table4.groupby(['Postcode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

df_table4 = pd.DataFrame(df_table4)
df_table4.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_table4.shape

(103, 3)

**df_table4 above is shape (103,3)** - submission for part 1 of peer review

## Part 2 - Concatinate initial table with Geospacial Coordinates

In [13]:
Geo = pd.DataFrame(Geospacial_Coordinates)
Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_table_final = pd.concat([df_table4, Geo], axis=1)
df_table_final = df_table_final.drop(['Postal Code'], axis = 1)
df_table_final.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3 - setup mapping

In [15]:
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [17]:
df_toronto = df_table_final[df_table_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Part 3a - Create a map using Borough names that include 'Toronto' & highlight each Neighborhood

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

### Part 3b - explore a neighborhood using Foursquare API

In [19]:
CLIENT_ID = 'AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1' # my Foursquare ID
CLIENT_SECRET = 'P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1
CLIENT_SECRET:P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E


**Let's explore the 'studio district'.. that sounds like a cool spot**

In [20]:
#define objects for 'Studio District' index [3] in df_toronto
neighborhood_latitude = df_toronto.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[3, 'Longitude'] # neighborhood longitude value
neighborhood_name = df_toronto.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Studio District are 43.6595255, -79.340923.


**Now, let's get the top 100 venues that are in Studio District within a radius of 500 meters.**

In [21]:
#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display GET request URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1&client_secret=P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E&v=20180605&ll=43.6595255,-79.340923&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results; # remove ';' to see json data

**clean the json and structure it into a *pandas* dataframe.**

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Leslieville Pumps,Sandwich Place,43.660892,-79.340626
2,Te Aro,Coffee Shop,43.661373,-79.338577
3,Queen Books,Bookstore,43.660651,-79.342267
4,Hooked,Fish Market,43.660407,-79.343257


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

39 venues were returned by Foursquare.


**create a map of the studio district and highlight nearby venues

In [26]:
map_studio = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=17)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_studio) 
    
map_studio

### Part 3c - Cluster Analysis of Venues across all neighborhoods

In [27]:
# create a function to get all venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
#run function for all toronto neighborhoods and create df 'toronto_venues'
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
Riverdale,The Danforth West
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Summerhill West,Forest Hill SE,Deer Park,South Hill,Rathnelly
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
Yorkville,The Annex,North Midtown
University of Toronto,Harbord
Kensington Market,Grange Park,Chinatown
South Niagara,Bathurst Quay,King and Spadina,Railway Lands,CN Tower,Harbourfront West,Island airport
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Exhibition Place,Parkdale Village,Brockton
High Park,The Junction South
Parkdale,Roncesvall

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1695, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale,The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [30]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
Central Bay Street,83,83,83,83,83,83
Christie,15,15,15,15,15,15


In [31]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 unique categories.


**create 'one hot' file with dummy values by venue category**

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,Richmond,King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.012048,0.000000,0.012048,0.000000,0.012048,0.000000,0.00
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Church and Wellesley,0.011364,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.011364,0.000000,0.011364,0.000000,0.011364,0.00
6,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00


In [34]:
toronto_grouped.shape

(38, 236)

**Let's print each neighborhood along with the top 5 most common venues**

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Richmond,King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Restaurant  0.05
3        Bakery  0.04
4   Cheese Shop  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2              Park  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4                 Bar  0.04


----Christie----
               venue  freq
0               Café  0.20
1      Grocery Store  0.20
2               Park  0.13
3  Convenience Store  0.07
4         Baby Store  0.07


----Church and Wellesley----
                

**First, let's write a function to sort the venues in descending order.**

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Pub
2,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar
4,Christie,Café,Grocery Store,Park,Nightclub,Convenience Store
5,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint
6,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant
7,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant
8,Davisville North,Burger Joint,Sandwich Place,Breakfast Spot,Gym,Grocery Store
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,American Restaurant


**cluster neighborhoods**

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([0, 0, 0, 0, 4], dtype=int32)

In [39]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Discount Store,Filipino Restaurant
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Italian Restaurant,Pet Store,Gym,Coffee Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Dim Sum Restaurant,Bus Line,Women's Store


In [40]:
toronto_merged.tail()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
33,M6K,West Toronto,"Exhibition Place,Parkdale Village,Brockton",43.636847,-79.428191,0,Breakfast Spot,Coffee Shop,Café,Performing Arts Venue,Burrito Place
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,0,Mexican Restaurant,Café,Bar,Bakery,Fried Chicken Joint
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,0,Breakfast Spot,Gift Shop,Restaurant,Dog Run,Burger Joint
36,M6S,West Toronto,"Swansea,Runnymede",43.651571,-79.484450,0,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant


## Final output - cluster map by venue

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters